![ALT_TEXT_FOR_SCREEN_READERS](./header.png)

# Exercise 4.D LLM Information Extraction

The goal of this exercise is to demonstrate the possibility of information extraction from textual information along two use cases:
- extracting personal information from email and prepare it for update of CRM system.
- extraction of relevant information from an Austrian Grundbuchauszug.


We are using **OLLAMA**[1] for local execution of the LLM and the framework **langchain**[2] for the access to the model.

- [1] [https://ollama.com/](https://ollama.com/)
- [2] [https://www.langchain.com/](https://www.langchain.com/)



# Considerations

- Read the tutorial carefully
- Install OLLAMA [1] on your machine
- Start with the model llama3.2 https://ollama.com/library/llama3.2 and try to serve it using ollama
- Use ```ollama pull <name-of-model>``` to load a model from the server to a local memory. Take care for the memory footprint on your machine.
- Install **langchain-ollama** and **langchain** packages into your environment (use pip inside the workbook and comment it out later)

# Requirements

- R0: Extend the contact information extraction by Address and phone number (30%)
- R1: Extend the Grundbuch information to show also Part C elements (40%)
- R2: Experiment with better online LLMs (30%)

# Setup

We are using ollama and langchain.

In [ ]:
#%pip install -U langchain-ollama

In [ ]:
#%pip install -U langchain

In [ ]:
#%pip install -qU langchain_mistralai

In [ ]:
#%pip install -qU pypdf

In [ ]:
#%pip install -qU "pydantic[email]"

# eMail to CRM 

A chat completion model takes a list of messages as input and generates the next message. The resulting message is of type **AIMessage**.
The list of messages can contain tuples with role and prompt for different message objects from the langchain_core.messages module.

In [ ]:
import os
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional
from langchain_core.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv


In [ ]:
llm = ChatOllama(
    model="llama3.2",
    temperature=0.0
)

In [ ]:
fake_email = """
Betreff: Rückfrage zu Ihrem Angebot

Sehr geehrte Damen und Herren,

vielen Dank für die Zusendung Ihres Angebots. Wir haben noch einige Rückfragen bezüglich der Zahlungsmodalitäten und würden uns über eine kurze Rückmeldung freuen.

Mit freundlichen Grüßen  
Dr. Martina Schneider

--  
Dr. Martina Schneider  
Leiterin Einkauf  
Global Procurement Department  
MediTech Solutions GmbH  
Hauptstraße 123  
12345 Berlin  
Telefon: +49 30 123456-789  
E-Mail: martina.schneider@meditech-solutions.de  
"""

In [ ]:
#
# Definition of the information to extract, the LLM reads the description to understand the meaning of the field
#
class ContactInfo(BaseModel):
    name: str = Field(..., description="Der vollständige Name der Person")
    position: Optional[str] = Field(None, description="Die Position oder Funktion der Person")
    department: Optional[str] = Field(None, description="Die Abteilung, in der die Person arbeitet")
    company: Optional[str] = Field(None, description="Der Name des Unternehmens")
    address: Optional[str] = Field(None, description="Die vollständige Geschäftsadresse")
    phone: Optional[str] = Field(None, description="Die geschäftliche Telefonnummer")
    email: Optional[EmailStr] = Field(None, description="Die geschäftliche E-Mail-Adresse")

In [ ]:
structured_llm = llm.with_structured_output(ContactInfo)

In [ ]:
extract = structured_llm.invoke(fake_email)

In [ ]:
extract

# Grundbuch Extraction

In [ ]:
#
# Definition of the information to extract, the LLM reads the description to understand the meaning of the field
#

from pydantic import BaseModel, Field, EmailStr, conint, constr
from typing import List, Optional, Union

In [ ]:

class Anteil(BaseModel):
    blnr: Optional[Union[int, str]] = Field(None, description="Laufende Nummer im B-Blatt (z. B. 31 oder '31')")
    name: Optional[str] = Field(None, description="Vollständiger Name des Eigentümers")
    geburtsdatum: Optional[str] = Field(None, description="Geburtsdatum im Format YYYY-MM-DD")
    adresse: Optional[str] = Field(None, description="Vollständige Wohnadresse (inkl. Straße, PLZ, Ort)")
    anteil: Optional[str] = Field(None, description="Anteil an der Liegenschaft (z. B. '200/4000')")
    wohnungseigentum_an: Optional[str] = Field(None, description="Bezeichnung der Wohnung (z. B. 'W 7'), sofern Wohnungseigentum besteht")
    rechtsgrund: Optional[str] = Field(None, description="Rechtsgrundlage für das Eigentum (z. B. 'Kaufvertrag 2024-02-15')")

class Belastung(BaseModel):
    lfd_nr: Optional[Union[int, str]] = Field(None, description="Laufende Nummer im C-Blatt (z. B. 18 oder '18')")
    art: Optional[str] = Field(None, description="Art der Belastung (z. B. 'Pfandrecht', 'Dienstbarkeit')")
    inhalt: Optional[str] = Field(None, description="Detaillierte Beschreibung der Belastung (z. B. Vertragstext oder Zweck)")
    betrag: Optional[str] = Field(None, description="Betrag bei Pfandrechten (inkl. Währung, z. B. 'EUR 300.000,--')")
    glaeubiger: Optional[str] = Field(None, description="Name des Gläubigers bei Pfandrechten")
    betroffene_blnrs: Optional[List[Union[int, str]]] = Field(None, description="Liste der betroffenen B-LNRs (z. B. [31, 32])")

class GrundbuchEintrag(BaseModel):
    einlagezahl: Optional[Union[str, int]] = Field(None, description="Einlagezahl der Liegenschaft (z. B. '6789')")
    katastralgemeinde: Optional[str] = Field(None, description="Bezeichnung der Katastralgemeinde inkl. Nummer (z. B. '35791 Sonnenfeld')")
    anteile: Optional[List[Union[Anteil, dict]]] = Field(None, description="Liste der Eigentumsanteile laut B-Blatt (****  B  ****)")
    belastungen: Optional[List[Union[Belastung, dict]]] = Field(None, description="Liste der Belastungen im C-Blatt (****  C  ****)")

    class Config:
        arbitrary_types_allowed = True


In [ ]:
#
# Einlesen eines Grundbuch PDFs
#

In [ ]:
#
# Define PDF file to read
#
file_path = "./documents/KATASTRALGEMEINDE 35791 Sonnenfeld EINLAGEZAHL 6789.pdf"

In [ ]:
#
# Load content of PDF file into memory
#
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader(file_path,mode="single")

In [ ]:
pages = loader.load()

In [ ]:
grundbuch_inhalt = ''
for page in pages:
    grundbuch_inhalt = grundbuch_inhalt + page.page_content

In [ ]:
print(grundbuch_inhalt)

In [ ]:
system_prompt = """

Du bist ein juristischer Assistent und extrahierst strukturierte Informationen aus Grundbuchauszügen aus Österreich. Achte auf das genaue Ausfüllen
der Anteil Liste und der Belastungen Liste.

Grundbucheintrag
-----------------------------------------------------
{grundbuch_inhalt}
-----------------------------------------------------

Analysiere bitte den Text im Grundbucheintrag und extrahiere alle Felder.

"""

In [ ]:
prompt = PromptTemplate.from_template(system_prompt)

In [ ]:
messages = prompt.invoke({"grundbuch_inhalt": grundbuch_inhalt})

In [ ]:
#
# Try alternative LLMs if the basic LLM does not work
#

In [ ]:
#
# R2: experiment with a better LLM, e.g. Mistral
#
#load_dotenv()
#api_key = os.environ["MISTRAL_API_KEY"]
#from langchain_mistralai import ChatMistralAI
#llm = ChatMistralAI(model="mistral-small-latest",temperature=0,max_retries=2,)

In [ ]:
structured_llm = llm.with_structured_output(GrundbuchEintrag)

In [ ]:
result = structured_llm.invoke(messages)

In [ ]:
result

In [ ]:
type(result)